In [ ]:
# Python 3.10.13
import time
import json
import re
import concurrent.futures

from selenium import webdriver
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

import utils
import selenium_scrappers as scrappers

# Definición de parámetros para la extracción

In [ ]:
# Listado de palabras a poner en la barra de busqueda
grupos = ['mineria ilegal']

# Criterios a buscar en cada noticia que aparezca en la busqueda
criterios = [' oro ', 'minería ilegal', 'minería clandestina']

carteles_map = {
    'CJNG': ['CJNG', 'cartel de jalisco nueva generacion', 'cártel de jalisco nueva generación', ],
    'CDS': ['CDS', 'Cartel de sinaloa', 'cártel de sinaloa','CDG', 'cartel del golfo', 'cártel del golfo', ],
    'CDN': ['CDN', 'cartel del noreste', 'cártel del noreste', ],
    'NFM': ['NFM', 'nueva familia michoacana', ]
}

carteles = [l for x in carteles_map.values() for l in x]

actividades = ['robo', 'extorsión', 'homicidio']

# Nombres de los archivos que guarda el scrapper
nombre = 'prueba_scrapper_1'
nombre_archivo_extraccion = f'{nombre}_raw' # file con titulo y link de cada noticia encontrada
nombre_archivo_filtrado = f'{nombre}_filtrado' # file con las noticias que cumplen los criterios 
nombre_indice = f'{nombre}_indice' # 2 indices son creados, uno expandido con el parrafo especifico donde se encontro el match y uno resumido listando los matches en cada noticia

municipios = pd.read_excel('extraction_files/municipios.xlsx', usecols=['ESTADO', 'MUNICIPIO'])

# Proceso de extracción de links
# Este proceso no es necesario en caso de backup

In [ ]:
try:
    browser = webdriver.Chrome(ChromeDriverManager().install())
except:
    browser = webdriver.Chrome()

In [ ]:
extracted_entries = {grupo: [] for grupo in grupos}
for key in extracted_entries.keys():
    current_links = []
    current_links.extend(scrappers.get_links_from_infobae(browser=browser, word=key))
    current_links.extend(scrappers.extract_links_from_universal(browser=browser, word=key))
    current_links.extend(scrappers.get_links_from_sinembargo(browser=browser, word=key))
    current_links.extend(scrappers.get_articulos_from_elsoldemexico(browser=browser, word=key))
    current_links.extend(scrappers.get_columnas_from_elsoldemexico(browser=browser, word=key))
    current_links.extend(scrappers.get_links_from_vanguardia(browser=browser, word=key))
    
    total_links = list(set(current_links))
    extracted_entries[key].extend(total_links)

## Guardar archivo con todos los links extraídos

In [ ]:
with open(f'extraction_files/{nombre_archivo_extraccion}.json', 'w') as f:
    json.dump(extracted_entries, f)

## Opcional: Cargar un archivo con links anteriormente guardado [cambiar nombre]

In [ ]:
# with open(f'extraction_files/{nombre_archivo_extraccion}.json', 'r') as f:
#     extracted_entries = json.load(f)

## Algoritmo de busqueda de criterios

### Para matar el proceso sin perder las variables seleccionar interrupt

In [ ]:
filtered_entries = {grupo: [('titulo', 'year', 'matching_criterios', 'matching_estados', 'matching_municipios', carteles, actividades, 'url'), ] for grupo in extracted_entries.keys()}

In [ ]:
for grupo, entries in extracted_entries.items():
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        future_to_url = {executor.submit(utils.get_matches, collector=filtered_entries,
                                         grupo=grupo, url=entry, criterios=criterios,
                                         municipios=municipios, actividades=actividades,
                                         carteles=carteles): entry
                         for entry in entries}
    

## Guardar el archivo con las coincidencias 

In [ ]:
with open(f'extraction_files/{nombre_archivo_filtrado}.json', 'w') as f:
    json.dump(filtered_entries, f)

## Opcional: Leer un archivo filtrado anteriormente guardado

In [ ]:
# with open(f'extraction_files/{nombre_archivo_filtrado}.json', 'r') as f:
#     filtered_entries = json.load(f)

## Generar excel con el formato deseado

In [ ]:
datasets = []
for k, v in filtered_entries.items():
    current_df = pd.DataFrame(data=v, columns=('titulo', 'year', 'matching_criterios',
                                               'matching_estados', 'matching_municipios',
                                               'matching_carteles', 'matching_actividades', 'url'
                                               ))
    current_df.insert(0, "busqueda", k)
    datasets.append(current_df)

In [ ]:
index_df = pd.concat(datasets, ignore_index=True)
index_df.drop_duplicates(subset=['url'], inplace=True)
index_df = index_df[index_df['titulo'] != 'titulo']
index_df[carteles] = index_df['matching_carteles'].to_list()
index_df[carteles] = index_df[carteles].astype(int)
index_df[actividades] = index_df['matching_actividades'].to_list()
index_df[actividades] = index_df[actividades].astype(int)
index_df.drop(columns=['matching_carteles', 'matching_actividades'], inplace=True)
index_df = index_df[['titulo', 'year', 'matching_criterios', 'matching_estados', 'matching_municipios'] + carteles + actividades + ['url', ]]
for k, v in carteles_map.items():
    index_df[k] = (index_df[v].sum(axis=1) > 0).astype(int)
    if k in v:
        index_df.drop(columns=list(set(v).symmetric_difference(set([k, ]))), inplace=True)
    else:
        index_df.drop(columns=v, inplace=True)
        
index_df.to_excel(f"extraction_files/{nombre_indice}.xlsx", index=False)